In [ ]:
# 데이터 구조
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
# 나무 구조 시각화
from sklearn.tree import plot_tree
# 데이터 분할: train, test
from sklearn.model_selection import train_test_split
# 분류 Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# 분류 xgboosting
!pip install xgboost
from xgboost import XGBClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
# 모델 성능 평가
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# 한글 맑은 고딕 적용
matplotlib.rc("font", family = "Malgun Gothic")
# 음수값 표시
matplotlib.rc("axes", unicode_minus = False)

In [ ]:
# 데이터 불러오기
df_raw = pd.read_csv("../data/bank.csv")
df_raw.head()

In [ ]:
# 결측치 확인
df_raw.isnull().sum(axis = 0)
# JOB 변수의 결측치는 "Other"로 대체
df_raw["JOB"].fillna("Other", inplace = True)
# 숫자형 변수의 결측치는 해당 변수의 평균값으로 대체
df_raw.fillna(df_raw.mean(), inplace=True)

In [ ]:
# get_dummies: 데이터의 문자형 변수에 대한 더미변수 생성
df_raw_dummy = pd.get_dummies(df_raw)
# 더미변수 생성된 데이터의 상위 5개 row를 확인
df_raw_dummy.head()

In [ ]:
# 학습용/평가용 데이터 분리
df_train, df_test = train_test_split(df_raw_dummy, test_size = 0.3, random_state = 1234)
print("학습용 데이터의 크기 : {}".format(df_train.shape))
print("평가용 데이터의 크기 : {}".format(df_test.shape))

In [ ]:
# 설명변수와 목표변수를 분리
df_train_y = df_train["BAD"]
df_train_x = df_train.drop("BAD", axis = 1, inplace = False)
df_test_y = df_test["BAD"]
df_test_x = df_test.drop("BAD", axis = 1, inplace = False)

In [ ]:
# 그래디언트 부스팅 모델 생성: GradientBoostingClassifier
gb_uncustomized = GradientBoostingClassifier(random_state=1234)
gb_uncustomized.fit(df_train_x, df_train_y)
# 학습용 데이터 셋 정확도
print("학습용 데이터의 정확도 : {:.3f}".format(gb_uncustomized.score (df_train_x, df_train_y)))
# 평가용 데이터 셋 정확도
print("평가용 데이터의 정확도 : {:.3f}".format(gb_uncustomized.score (df_test_x, df_test_y)))

In [ ]:
estimator = GradientBoostingClassifier(random_state=1234)
# 구하고자 하는 parameter와 범위
param_grid = {"learning_rate": [0.1, 0.2, 0.3], "n_estimators": [100, 150, 200], "max_depth": [4, 5, 6, 7], "min_samples_leaf": [1, 2, 3, 4, 5]}
# 정확도가 높은 최적 parameter 찾기
grid_gb = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_gb.fit(df_train_x, df_train_y)
print("best estimator model: \n{}".format(grid_gb.best_estimator_))
print("\nbest parameter: \n{}".format(grid_gb.best_params_))
print("\nbest score: \n{}".format(grid_gb.best_score_.round(3)))

In [ ]:
# 최종 모델
gb_final = GradientBoostingClassifier(min_samples_leaf = 5, max_depth = 6, n_estimators = 200,
learning_rate = 0.3, random_state=1234)
gb_final.fit(df_train_x, df_train_y)
# 예측
y_pred = gb_final.predict(df_test_x)
# 학습용 데이터 셋 정확도
print("학습용 데이터의 정확도 : {:.3f}".format(gb_final.score(df_train_x, df_train_y)))
# 평가용 데이터 셋 정확도
print("평가용 데이터의 정확도 : {:.3f}".format(gb_final.score(df_test_x, df_test_y)))
# confusion matrix
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))

In [ ]:
# Classification Report
print("Classification Report : ", classification_report(df_test_y, y_pred))

In [ ]:
# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance ["Feature"] = df_train_x.columns
df_importance ["Importance"] = gb_final.feature_importances_
# df_feature_importance의 테이블을 중요도 순으로 정렬
df_importance .sort_values("Importance", ascending=False, inplace = True)
df_importance .round(3)

In [ ]:
# 설명변수 중요도 그래프
df_importance .sort_values("Importance", ascending=True, inplace = True)
coordinates = range(len(df_importance ))
plt.barh(y = coordinates, width = df_importance ["Importance"])
plt.yticks(coordinates, df_importance ["Feature"])
plt.xlabel("설명변수 중요도")
plt.ylabel("설명변수")

In [ ]:
# 기본 모델 생성
xgb_uncustomized = XGBClassifier(random_state=1234)
xgb_uncustomized.fit(df_train_x, df_train_y)
# 학습용 데이터 셋 정확도
print("학습용 데이터의 정확도 : {:.3f}".format(xgb_uncustomized.score (df_train_x, df_train_y)))
# 평가용 데이터 셋 정확도
print("평가용 데이터의 정확도 : {:.3f}".format(xgb_uncustomized.score (df_test_x, df_test_y)))

In [ ]:
xgb_uncustomized